In [1]:
print("test")

test


In [2]:
import re
sample_file_name = "../../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../../dutch-language/data/corrected-sentences-nlwiki.txt"

file_name = corrected_file_name

regex_worden = re.compile(r'(.+ w|W)ord\b.*')
with open(file_name) as file:
    total_count = 0
    match_count = 0

    for line in file:
        if (total_count >= 2500000):
            break
        total_count += 1
        clean_line = line.rstrip()
        if (regex_worden.match(clean_line)):
            match_count += 1
            print(clean_line)

    print(total_count)
    print(match_count)

Daar word ik hartstikke gek van.
Ik hoop dat dit dus niet bij jou gedaan word.
Zo verving von Baader Descartes' cogito ergo sum door cogitor ergo sum: ik word gedacht, dus ik ben.
Ik ga door tot Parijs en zal winnen, tenzij ik vermoord word.
Hij zou op de dag van de verkiezing tegen zijn moeder gezegd hebben Of ik keer terug als pontiflex, of ik word voor altijd verbannen.
Rond het thema van de Vlaamse onafhankelijkheid schreef hij ‘Operatie Vlaamse Onafhankelijkheid‘, een verslag van de evaluatiedag ‘Volk, word staat’ dat op 24 november 2007 plaatsvond in het Federale Parlement.
Als je goed leeft, kun je na de dood in een hogere kaste terecht komen en uiteindelijk word je bevrijd van het aardse leven om samen te leven met god.
Vertaling: Ik word beschermd of hij beschermt mij.
Als je van deze appels eet, word je onsterfelijk.
In 2008 begon BNN met een grootse campagne onder het motto 'Koop dit, word lid'.
Nu pas word ik me ervan bewust dat mijn werk in zwart, wit en kleine kleurvlakke

In [11]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "bert", "outputs/", use_cuda=True
)
print(type(model))


<class 'simpletransformers.classification.classification_model.ClassificationModel'>


In [20]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",
])
print()
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 1/1 [00:00<00:00,  3.50it/s]
[0 0 0 0 1 1 1 1]
[[ 5.028973  -4.6238155]
 [ 5.015533  -4.553905 ]
 [ 5.001874  -4.5051813]
 [ 4.8724637 -4.4218674]
 [-5.3947897  4.4733806]
 [-5.344915   4.23907  ]
 [-5.315748   4.102456 ]
 [-5.1123905  3.8351293]]

